In [1]:
from functions import *

In [2]:
max_ = -1
for k,v in mapping.items():
    if v>max_:
        k_max = k
        max_ = v

In [4]:
evaluate(matrix[:,np.array(k_max)],np.array(k_max),policy_lookup,len(k_max))

2.1029411764705874

In [4]:
max_

2.1029411764705874

In [2]:
from functions_pt import *

In [3]:
embed_size = 128
dropout = 0.1
multiple_factor = 1
layers = 2
batch_size = 64
agg = 'mean' # mean,sum,min,max
model = pointNet(layers,embed_size,agg,dropout,multiple_factor)#.to('cuda')
model.load_state_dict(torch.load('baseline0.pt'))
model.eval()
model = model.to('cuda')

In [4]:
words_embed = np.load('possible_words_embed.npy')

In [8]:
# slightly worse than # of steps required by mapping as expected
evaluate(matrix[:,np.array(k_max)],np.array(k_max),policy_model,len(k_max),model=model,words_embed=words_embed,top=0.6)

2.1249999999999987

In [5]:
# see how well model does on full problem that it is not trained on
#     if count == 1: return 1
#     if count == 2: return 1.5
evaluate(matrix,np.arange(2309),policy_model,2309,model=model,words_embed=words_embed,top=0.6)

3.5036812472932013